In [1]:
import bs4
import requests
import pandas as pd
import time
from datetime import date

from bs4 import BeautifulSoup

### Automate Scraping NBA Top Shot Sales Data from Website (https://nbatopshot.com/transactions)

In [2]:
def getTopShotData():
    
    r = requests.get('https://nbatopshot.com/transactions')

    soup = bs4.BeautifulSoup(r.text, "xml")
        
    name = soup.find_all('td',{'class' : 'TableStyles__TD-sc-1saikeo-9 TableStyles__TDPlayerName-sc-1saikeo-14 bjxMhG bYjjiV'})

    price = soup.find_all('td', {'class':'TableStyles__TD-sc-1saikeo-9 TableStyles__TDPrice-sc-1saikeo-15 bjxMhG sCKOh'})

    buyer = soup.find_all('td', {'class':'TableStyles__TD-sc-1saikeo-9 TableStyles__TDUser-sc-1saikeo-17 TableStyles__TDBuyer-sc-1saikeo-18 bjxMhG gQiuXL bZQvYg'})

    seller = soup.find_all('td', {'class':'TableStyles__TD-sc-1saikeo-9 TableStyles__TDUser-sc-1saikeo-17 bjxMhG gQiuXL'})

    date_time = soup.find_all('td', {'class':'TableStyles__TD-sc-1saikeo-9 TableStyles__TDTime-sc-1saikeo-19 bjxMhG gZGvwR'})

    other = soup.find_all('td', {'class':'TableStyles__TD-sc-1saikeo-9 bjxMhG'})

    dat = pd.DataFrame()
    
    for i in range(50):
        dat.loc[i,0] = name[i].text
        dat.loc[i,1] = price[i].text
        dat.loc[i,2] = buyer[i].text
        dat.loc[i,3] = seller[i].text
        dat.loc[i,4] = date_time[i].text
    j = 0
    for i in range(200):
        if (i%4) == 0:
            dat.loc[j,5] = other[i].text
        elif (i%4) == 1:
            dat.loc[j,6] = other[i].text
        elif (i%4) == 2:
            dat.loc[j,7] = other[i].text
        else:
            j += 1
    
    dat.columns = ['Name','Price','Buyer','Seller','Date Time','Serial','Set','Series']
    dat = dat.iloc[::-1].reset_index(drop=True)
    return dat


def getLots(dat):
    today_date = date.today().strftime("%b-%d-%Y")
    timing = time.strftime("%H:%M:%S", time.localtime())
    start = time.time()
    while (time.time() - start) < 900:
        dat2 = getTopShotData()
        dat = pd.concat([dat,dat2]).drop_duplicates().reset_index(drop=True)

    dat['Price'] = dat['Price'].str.replace('$','').str.replace(',','').astype(float).astype(int)
    dat.to_csv(f'/Users/benedictjung/Desktop/NBA_DATA/sales_{today_date}_{timing}.csv',index=False)

    return dat

nba_dat = getTopShotData()
nba_dat = getLots(nba_dat)

In [3]:
nba_dat

,Name,Price,Buyer,Seller,Date Time,Serial,Set,Series
0,Mason Plumlee,2,@TheOneHitWonder,@MrMuckduck,"Aug 24, 21 11:37 PM",Common #36818/40000LE,Base Set,2
1,Jeremy Lamb,5,@Jackymoon,@Badger2424,"Aug 24, 21 11:37 PM",Common #9377/12000LE,Base Set,2
2,Skylar Mays,19,@Talent2fresh,@bug_nuts,"Aug 24, 21 11:37 PM",Common #4988/12000LE,Base Set,2
3,Chris Paul,9,@jordanthegod,@Anderson_Pack,"Aug 24, 21 11:37 PM",Common #7808/10500LE,2021 NBA Playoffs,2
4,Marcus Smart,5,@Magick,@TADPod,"Aug 24, 21 11:37 PM",Common #7755/12000LE,2021 NBA Playoffs,2
...,...,...,...,...,...,...,...,...
680,Jalen Brunson,3,@TPOPE,@SickoSkulz,"Aug 24, 21 11:53 PM",Common #31360/40000LE,Base Set,2
681,Isaac Okoro,6,@Sketti420,@big_boi,"Aug 24, 21 11:53 PM",Fandom #26054/27112+CC,Summer of '21,Summer 2021
682,Khris Middleton,10,@Uppityy,@jeanschoen,"Aug 24, 21 11:53 PM",Common #7299/15000LE,Base Set,2
683,RJ Barrett,7,@Yeti416,@Grae,"Aug 24, 21 11:53 PM",Common #9634/12000LE,2021 NBA Playoffs,2


### Automate scraping Lowest Ask Prices Data on Every Top Shot Moments (https://momentranks.com/topshot/moments)

In [4]:
def getLPData():
    
    all_players = [
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=2020%20NBA%20Finals',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=2021%20All-Star%20Game',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=2021%20NBA%20Finals',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=2021%20NBA%20Playoffs',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=And%20Then%20There%20Were%20Four',
        'https://momentranks.com/topshot/moments?challenge=&limit=1000&series=&setNames=Base%20Set',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Conference%20Semifinals',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Cool%20Cats',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Cosmic',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Deck%20the%20Hoops',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Denied%21',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Early%20Adopters',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Eastern%20Conference%20Finals',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=First%20Round',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=For%20the%20Win',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=From%20the%20Top',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Got%20Game',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Holo%20Icon',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Holo%20MMXX',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Hometown%20Showdown%3A%20Cali%20vs.%20NY',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Hustle%20and%20Show',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Lace%20%27Em%20Up',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=MVP%20Moves',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Metallic%20Gold%20LE',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Rising%20Stars',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Rookie%20Debut',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Run%20It%20Back',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Season%20Tip-off',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Seeing%20Stars',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=So%20Fresh',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Summer%20of%20%2721',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=The%20Finals',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=The%20Gift',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Throwdowns',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=Western%20Conference%20Finals',
        'https://momentranks.com/topshot/moments?challenge=&limit=250&series=&setNames=With%20the%20Strip'
    ]
    
    timing = time.strftime("%H-%M-%S", time.localtime())
    final_dat = pd.DataFrame()
    
    for i in all_players:
        r = requests.get(i)

        soup = bs4.BeautifulSoup(r.text, "xml")

        name = soup.find_all('span',{'class' : 'MuiTypography-root MomentCard_name__2XUpm MuiTypography-caption MuiTypography-noWrap'})

        description = soup.find_all('span',{'class' : 'MuiTypography-root MuiTypography-caption MuiTypography-noWrap MuiTypography-gutterBottom'})

        lowest_ask = soup.find_all('p', {'class':'MuiTypography-root MomentCard_low__2LZdQ MuiTypography-body2'})

        dat = pd.DataFrame()
        for i in range(len(name)):
            dat.loc[i,0] = name[i].text
            dat.loc[i,1] = lowest_ask[i].text
        j = 0
        for i in range(len(description)):
            if (i%2) == 0:
                dat.loc[j,2] = description[i].text
            else:
                dat.loc[j,3] = description[i].text
                j += 1
        dat.columns = ['Name','Lowest Ask','Set','Serial_Rarity']
        final_dat = pd.concat([final_dat,dat]).reset_index(drop = True)
        
    final_dat['Lowest Ask'] = final_dat['Lowest Ask'].str.replace('Lowest Ask: ', '').str.replace('$','').str.replace(',','').astype(int)
    
    return final_dat

In [5]:
lowest_ask_dat = getLPData()

In [6]:
lowest_ask_dat

,Name,Lowest Ask,Set,Serial_Rarity
0,LeBron James,88000,2020 NBA Finals (Series 1),#/79 • Legendary
1,Kentavious Caldwell-Pope,8881,2020 NBA Finals (Series 1),#/33 • Legendary
2,Anthony Davis,7603,2020 NBA Finals (Series 1),#/79 • Legendary
3,Tyler Herro,5500,2020 NBA Finals (Series 1),#/79 • Legendary
4,Jimmy Butler,4995,2020 NBA Finals (Series 1),#/79 • Legendary
...,...,...,...,...
1874,Danilo Gallinari,169,With the Strip (Series 1),#/499 • Rare
1875,Josh Jackson,155,With the Strip (Series 1),#/499 • Rare
1876,Gary Harris,149,With the Strip (Series 1),#/499 • Rare
1877,Ish Smith,145,With the Strip (Series 1),#/499 • Rare


In [7]:
lowest_ask_dat[lowest_ask_dat['Name'] == "Anthony Edwards"]

,Name,Lowest Ask,Set,Serial_Rarity
200,Anthony Edwards,617,Base Set (Series 2),#/4000 • Common
483,Anthony Edwards,39,Base Set (Series 2),#/40000 • Common
1168,Anthony Edwards,10500,Holo Icon (Series 2),#/99 • Legendary
1414,Anthony Edwards,1699,Metallic Gold LE (Series 2),#/499 • Rare
1629,Anthony Edwards,2450,Rising Stars (Series 2),#/1040 • Rare
1799,Anthony Edwards,349,Throwdowns (Series 2),#/1800 • Rare


In [8]:
lowest_ask_dat[lowest_ask_dat['Name'] == "LeBron James"]

,Name,Lowest Ask,Set,Serial_Rarity
0,LeBron James,88000,2020 NBA Finals (Series 1),#/79 • Legendary
12,LeBron James,997,2021 All-Star Game (Series 2),#/2021 • Rare
46,LeBron James,170,2021 NBA Playoffs (Series 2),#/12000 • Common
168,LeBron James,17000,Base Set (Series 1),#/1000 • Common
169,LeBron James,14000,Base Set (Series 1),#/1000 • Common
170,LeBron James,7250,Base Set (Series 1),#/1000 • Common
176,LeBron James,2499,Base Set (Series 1),#/3113 • Common
206,LeBron James,490,Base Set (Series 2),#/7500 • Common
216,LeBron James,377,Base Set (Series 2),#/15000 • Common
452,LeBron James,46,Base Set (Series 2),#/35000 • Common
